In [1]:
# Dependencies
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
engine = create_engine("sqlite:///db.sqlite")

# Countries

Loads from a CSV copy of all [ISO country codes](https://www.iso.org/obp/ui/#search)

In [3]:
df = pd.read_csv("iso_countries.csv")

In [4]:
df.drop(columns=['French short name', 'Numeric'], inplace=True)

In [5]:
df.columns = ['short_name', 'code2', 'code3']

In [6]:
df

,short_name,code2,code3
0,Afghanistan,AF,AFG
1,Albania,AL,ALB
2,Algeria,DZ,DZA
3,American Samoa,AS,ASM
4,Andorra,AD,AND
...,...,...,...
244,Western Sahara*,EH,ESH
245,Yemen,YE,YEM
246,Zambia,ZM,ZMB
247,Zimbabwe,ZW,ZWE


In [7]:
df.to_sql('countries', engine, if_exists='replace')

# Population

In [8]:
df = pd.read_csv("./resources/population.csv", skiprows=4)
df.drop(columns=['Country Name', 'Indicator Name', 'Indicator Code', '2021', 'Unnamed: 66'], inplace=True)
# unpivot
df = pd.melt(df, id_vars="Country Code").dropna()

In [11]:
df.columns = ['country_code3', 'year', 'value']
df

,country_code3,year,value
0,ABW,1960,54208.0
1,AFE,1960,130836765.0
2,AFG,1960,8996967.0
3,AFW,1960,96396419.0
4,AGO,1960,5454938.0
...,...,...,...
16221,XKX,2020,1790133.0
16222,YEM,2020,29825968.0
16223,ZAF,2020,59308690.0
16224,ZMB,2020,18383956.0


In [12]:
df.to_sql('population', engine, if_exists='replace')

# Inflation

In [14]:
df = pd.read_csv("./resources/inflation.csv", skiprows=4)
df.drop(columns=['Country Name', 'Indicator Name', 'Indicator Code', '2021', 'Unnamed: 66'], inplace=True)
# unpivot
df = pd.melt(df, id_vars="Country Code").dropna()

In [15]:
df.columns = ['country_code3', 'year', 'value']
df

,country_code3,year,value
13,AUS,1960,3.728814
14,AUT,1960,1.945749
17,BEL,1960,0.299467
19,BFA,1960,7.788162
28,BOL,1960,11.533153
...,...,...,...
16220,WSM,2020,-1.568912
16221,XKX,2020,0.198228
16223,ZAF,2020,3.210036
16224,ZMB,2020,15.732585


In [16]:
df.to_sql('inflation', engine, if_exists='replace')